# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [20]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

Setting path to train and test data.

In [5]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32

Loading train data.

In [6]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [7]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [8]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [9]:
labels, images = load_train_data(TRAIN_DATA_DIR)
test_labels, test_images = load_test_data(TEST_DATA_DIR)

In [10]:
print("Total training images {0}".format(len(images)))
print("Total testing images {0}".format(len(test_images)))

Total training images 42890
Total testing images 12630


Creating bar chart for dataset Visualization.

In [ ]:
data = {x:labels.count(x) for x in labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel("Number of examples")
plt.title("Training dataset")
plt.show()

Resize and normalize images.

In [ ]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

Difference between normal image and a normalized one.

In [ ]:
plt.figure(figsize = (10, 10))
plt.subplot(1, 2, 1)
plt.axis('off')
plt.title("Normal image")  
plt.imshow(images[1800])
plt.subplot(1, 2, 2)
plt.axis('off')
plt.title("Normalized image")
plt.imshow(normalize(images[1800]))
plt.show()

In [11]:
with tf.device('/device:GPU:0'):
    images_ph = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3], name = "images_ph")
    labels_ph = tf.placeholder(tf.int64, [None])

    images_flat = tf.contrib.layers.flatten(images_ph)

    logits = tf.contrib.layers.fully_connected(images_flat, len(set(labels)), tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1, name="prediction")
   
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels_ph)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels_ph)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()

In [12]:
saver = tf.train.Saver()
session = tf.Session()
session.run(init)

In [14]:
labels_arr = np.array(labels)
images_arr = np.array(images)

In [15]:
feed_dict_train = {images_ph: images_arr,
             labels_ph: labels_arr}
feed_dict_test = {images_ph: test_images,
             labels_ph: test_labels}

In [17]:
for i in range(3001):
    loss_value = session.run([optimizer, loss], feed_dict_train)
    if i % 10 == 0:
        print("Loss: ", loss_value)

score = session.run(accuracy, feed_dict_test)
print("Accuracy: {:.3f}".format(score))

('Loss: ', [None, 3.918751])
('Loss: ', [None, 3.1980357])
('Loss: ', [None, 2.9128506])
('Loss: ', [None, 2.6995668])
('Loss: ', [None, 2.5396926])
('Loss: ', [None, 2.4162591])
('Loss: ', [None, 2.3168328])
('Loss: ', [None, 2.2327266])
('Loss: ', [None, 2.1568301])
('Loss: ', [None, 2.0883188])
('Loss: ', [None, 2.0276327])
('Loss: ', [None, 1.9736047])
('Loss: ', [None, 1.9248583])
('Loss: ', [None, 1.8806278])
('Loss: ', [None, 1.8401241])
('Loss: ', [None, 1.79357])
('Loss: ', [None, 1.7509197])
('Loss: ', [None, 1.7145026])
('Loss: ', [None, 1.6817114])
('Loss: ', [None, 1.6520028])
('Loss: ', [None, 1.6243144])
('Loss: ', [None, 1.5974531])
('Loss: ', [None, 1.5727818])
('Loss: ', [None, 1.5494262])
('Loss: ', [None, 1.5273693])
('Loss: ', [None, 1.5065006])
('Loss: ', [None, 1.4866126])
('Loss: ', [None, 1.4673597])
('Loss: ', [None, 1.4483916])
('Loss: ', [None, 1.4303248])
('Loss: ', [None, 1.4132693])
('Loss: ', [None, 1.3967917])
('Loss: ', [None, 1.3715771])
('Loss: ', [N

('Loss: ', [None, 0.55786604])
('Loss: ', [None, 0.55699766])
('Loss: ', [None, 0.55613327])
('Loss: ', [None, 0.55527312])
('Loss: ', [None, 0.55441707])
('Loss: ', [None, 0.55356508])
('Loss: ', [None, 0.55271715])
('Loss: ', [None, 0.55187327])
('Loss: ', [None, 0.55103314])
('Loss: ', [None, 0.55019706])
('Loss: ', [None, 0.54936486])
('Loss: ', [None, 0.54853648])
('Loss: ', [None, 0.54771197])
('Loss: ', [None, 0.54689109])
('Loss: ', [None, 0.54607403])
('Loss: ', [None, 0.54526061])
('Loss: ', [None, 0.54445088])
('Loss: ', [None, 0.54364473])
('Loss: ', [None, 0.54284215])
('Loss: ', [None, 0.54204303])
('Loss: ', [None, 0.54124755])
('Loss: ', [None, 0.54045558])
('Loss: ', [None, 0.53966695])
('Loss: ', [None, 0.53888202])
('Loss: ', [None, 0.53810036])
('Loss: ', [None, 0.53732222])
('Loss: ', [None, 0.53655469])
('Loss: ', [None, 0.53585738])
('Loss: ', [None, 0.53501284])
('Loss: ', [None, 0.53425729])
('Loss: ', [None, 0.53349131])
('Loss: ', [None, 0.53273654])
('Loss: 

In [28]:
saver.save(session, './model/model')

'./model/model'

In [ ]:
session.close()